# Q&A against a SQL Database

Now that we know (from the prior Notebook) how to query tabular data on a CSV file, let's try now to keep the data at is source and ask questions directly to a SQL Database.
The goal of this notebook is to demonstrate how a LLM so advanced as GPT-4 can understand a human question and translate that into a SQL query to get the answer. 

We will be using the Azure SQL Server that you created on the initial deployment. The server should be created on the Resource Group where the Azure Cognitive Search service is located.

Let's begin..

In [1]:
import os
import pandas as pd
import pyodbc
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain import SQLDatabaseChain
from langchain.chains import SQLDatabaseSequentialChain
from langchain.agents import AgentExecutor
from langchain.callbacks.manager import CallbackManager

from common.callbacks import MyCustomHandler
from common.prompts import MSSQL_PROMPT, MSSQL_AGENT_PREFIX, MSSQL_AGENT_FORMAT_INSTRUCTIONS

from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("credentials2.env")

def printmd(string):
    display(Markdown(string))

In [2]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"]
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"]
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]
os.environ["OPENAI_API_TYPE"] = "azure"

# Install MS SQL DB driver in your machine

We need the driver installed on this compute in order to talk to the SQL DB, so run the below cell once<br>
Reference: https://learn.microsoft.com/en-us/sql/connect/odbc/linux-mac/installing-the-microsoft-odbc-driver-for-sql-server?view=sql-server-ver16&tabs=ubuntu18-install%2Calpine17-install%2Cdebian8-install%2Credhat7-13-install%2Crhel7-offline

In [3]:
# !sudo ./download_odbc_driver.sh

# Load Azure SQL DB with the Covid Tracking CSV Data

The Azure SQL Database is currently empty, so we need to fill it up with data. Let's use the same data on the Covid CSV filed we used on the prior Notebook, that way we can compare results and methods. 
For this, you will need to type below the credentials you used at creation time.

In [39]:
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

db_config = {
                'drivername': 'mssql+pyodbc',
                'username': os.environ["SQL_SERVER_USERNAME"] +'@'+ os.environ["SQL_SERVER_ENDPOINT"],
                'password': os.environ["SQL_SERVER_PASSWORD"],
                'host': os.environ["SQL_SERVER_ENDPOINT"],
                'port': 1433,
                'database': os.environ["SQL_SERVER_DATABASE"],
                'query': {'driver': 'ODBC Driver 18 for SQL Server'}
            }

# Create a URL object for connecting to the database
db_url = URL.create(**db_config)

# Print the resulting URL string
# print(db_url)

# Connect to the Azure SQL Database using the URL string
engine = create_engine(db_url)

# Test the connection
try:
    conn = engine.connect()
    print("Connection successful!")
    result = engine.execute("SELECT @@Version")
    for row in result:
        print(row)
    conn.close()
    
except OperationalError:
    print("Connection failed.")

Connection successful!
('Microsoft SQL Azure (RTM) - 12.0.2000.8 \n\tMay 22 2023 22:22:02 \n\tCopyright (C) 2022 Microsoft Corporation\n',)


In [5]:
df = pd.read_csv("./data/updatedbunnsfullstockreport.csv").fillna(value = 0)
print("Rows and Columns:",df.shape)
df.head()

Rows and Columns: (41951, 179)


,Id,CountryCode,Fineline,Team_Member,Supersession_Next_Item,Supersession_Previous_Item,DeptName,Sub_Dept,Item_Description,Item_Band,...,DC_Weeks_On_Hand_01_Apr,DC_Weeks_On_Hand_08_Apr,DC_Weeks_On_Hand_15_Apr,DC_Weeks_On_Hand_22_Apr,DC_Weeks_On_Hand_29_Apr,DC_Weeks_On_Hand_06_May,DC_Weeks_On_Hand_13_May,DC_Weeks_On_Hand_20_May,DC_Weeks_On_Hand_27_May,Next_Order_Due_Date
0,0,1280NZ,1280,Team Member 1,0.0,0.0,300 TOOL ACCESSORIES,400 DRILLING,NESTED DRILL/DRVR++SETS 6360177-180,Band E,...,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,0
1,1,1380NZ,1380,Team Member 2,0.0,0.0,300 CLEANING AND ACCESSORIES,400 CLEANING ACCESSORIES,NESTED CLOTHS++4460819-820 & 823,Band E,...,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,0
2,2,1400NZ,1400,Team Member 3,0.0,0.0,300 TOOL ACCESSORIES,400 DRILLING,NESTED BIT SET/DRILL++222506/511/514/517,Band E,...,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,0
3,3,1430NZ,1430,Team Member 4,0.0,0.0,300 FURNITURE,400 FURNITURE ACCESSORIES,NESTED CUSHION ASSTD++3191892/894/895,Band E,...,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,0
4,4,1440NZ,1440,Team Member 5,0.0,0.0,300 FURNITURE,400 FURNITURE ACCESSORIES,NESTED CUSHION THROW++3192085/086,Band E,...,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,0


In [ ]:
df.columns

In [7]:
#df.to_sql('bunnsfullstockrep', con=engine, if_exists='replace', index=False)

-1

In [12]:
column_names = df.rename(columns={
    'Stock_Unit_Cost_Zone1_(ExTax)_(based_on_last_cost_product_landed_at)': 'Stock_Unit_Cost_Zone1_ExTax_based_on_last_cost_product_landed_at',
    'SOH_Units_in_DC_(inc._Rec.)': 'SOH_Units_in_DC_inc_Rec',
    'Store_Orders_=_Pending_Ords_+_On_Pick': 'Store_Orders_count_as_Pending_Ords_plus_On_Pick',
    'Sales_Units_MTD_%': 'Sales_Units_MTD_percent'}, inplace=True)

In [16]:
column_names = df.rename(columns={
    '$_in_DC' : 'USD_in_DC',
    '$_in_Stores': 'USD_in_Stores',
    '$_In_Transit': 'USD_In_Transit'},inplace=True)
    

In [34]:
column_names = df.rename(columns={
    '3PL_SOH':'ThirdPartyLogistics_SOH',
     '8_Weeks_COGNOS_Sales': 'Eight_Weeks_COGNOS_Sales',
     '_DC_Weeks_On_Hand_09_Jul': 'DC_Weeks_On_Hand_09_Jul',
     '12_Month_Forecast': 'Twelve_Month_Forecast'
     },inplace=True)

In [26]:
df.columns = df.columns.str.replace('/', '_')

In [40]:
 #Infer column names and data types
column_names = df.columns.tolist()
column_types = df.dtypes.to_dict()
column_names, column_types


(['Id',
  'CountryCode',
  'Fineline',
  'Team_Member',
  'Supersession_Next_Item',
  'Supersession_Previous_Item',
  'DeptName',
  'Sub_Dept',
  'Item_Description',
  'Item_Band',
  'Planning_Status',
  'StatusDC',
  'State',
  'Ethical_Sourcing_Approved',
  'DC',
  'Transfer_Pack',
  'Units_per_Pallet',
  'Total_Min',
  'Total_Max',
  'Number_of_stores_ranged_for_the_item',
  'Shipping_Terms',
  'Supplier_Name',
  'Stock_Unit_Cost_Zone1_ExTax_based_on_last_cost_product_landed_at',
  'Metro_Retail',
  'Margin',
  'Qty_20ft',
  'Qty_40ft',
  'Qty_40hq',
  'MOQ',
  'SOH_Units_In_Store',
  'SOH_Units_In_Transit',
  'SOH_Units_in_DC_inc_Rec',
  'SOH_units_DC_Avail',
  'Store_Orders_count_as_Pending_Ords_plus_On_Pick',
  'Network',
  'ThirdPartyLogistics_SOH',
  'Sell_Through',
  'Pallet_Count_in_Store',
  'Pallet_Count_In_Transit',
  'Pallet_Count_in_DC',
  'Pallet_Count_in_Network',
  'WOH_future_forecast_in_Store',
  'WOH_future_forecast__in_DC',
  'WOH_future_forecast__in_Network',
  '

In [36]:
# Generate SQL statement to create table. We need this to match the types, otherwise everything will be a string and you wont be able to calculations 
table_name = 'bunnsfullstockrep3'

create_table_sql = f"CREATE TABLE {table_name} ("
for name, dtype in column_types.items():
    if dtype == 'object':
        create_table_sql += f"{name} VARCHAR(MAX), "
    elif dtype == 'int64':
        create_table_sql += f"{name} INT, "
    elif dtype == 'float64':
        create_table_sql += f"{name} FLOAT, "
    elif dtype == 'bool':
        create_table_sql += f"{name} BIT, "
    elif dtype == 'datetime64[ns]':
        create_table_sql += f"{name} DATETIME, "
create_table_sql = create_table_sql[:-2] + ")"


In [37]:

try:
    #Createse the table in Azure SQL
    engine.execute(create_table_sql)
    print("Table",table_name,"succesfully created")
except Exception as e:
    print(e)

Table bunnsfullstockrep3 succesfully created


In [41]:
# Insert data into the table in batches of 1000 rows
try:
    lower = 0
    upper = 1000
    limit = df.shape[0]

    while lower < limit:
        df[lower:upper].to_sql(table_name, con=engine, if_exists='append', index=False)
        print("rows:", lower, "-", upper, "inserted")
        lower = upper
        upper = min(upper + 1000, limit)

except Exception as e:
    print(e)

rows: 0 - 1000 inserted
rows: 1000 - 2000 inserted
rows: 2000 - 3000 inserted
rows: 3000 - 4000 inserted
rows: 4000 - 5000 inserted
rows: 5000 - 6000 inserted
rows: 6000 - 7000 inserted
rows: 7000 - 8000 inserted
rows: 8000 - 9000 inserted
rows: 9000 - 10000 inserted
rows: 10000 - 11000 inserted
rows: 11000 - 12000 inserted
rows: 12000 - 13000 inserted
rows: 13000 - 14000 inserted
rows: 14000 - 15000 inserted
rows: 15000 - 16000 inserted
rows: 16000 - 17000 inserted
rows: 17000 - 18000 inserted
rows: 18000 - 19000 inserted
rows: 19000 - 20000 inserted
rows: 20000 - 21000 inserted
rows: 21000 - 22000 inserted
rows: 22000 - 23000 inserted
rows: 23000 - 24000 inserted
rows: 24000 - 25000 inserted
rows: 25000 - 26000 inserted
rows: 26000 - 27000 inserted
rows: 27000 - 28000 inserted
rows: 28000 - 29000 inserted
rows: 29000 - 30000 inserted
rows: 30000 - 31000 inserted
rows: 31000 - 32000 inserted
rows: 32000 - 33000 inserted
Can't reconnect until invalid transaction is rolled back. (Backgr

In [58]:
#conn = engine.connect()
cursor = conn.cursor()
cursor.execute('SELECT TOP 2 * FROM table_name')
rows = cursor.fetchall()

for row in rows:
    print(row)


AttributeError: 'Connection' object has no attribute 'cursor'

In [ ]:
# # Read CSV file into a pandas dataframe
# csv_path = "./data/all-states-history.csv"
# df = pd.read_csv(csv_path).fillna(value = 0)

# # Infer column names and data types
# column_names = df.columns.tolist()
# column_types = df.dtypes.to_dict()

# # Generate SQL statement to create table
# table_name = 'covidtracking'

# create_table_sql = f"CREATE TABLE {table_name} ("
# for name, dtype in column_types.items():
#     if dtype == 'object':
#         create_table_sql += f"{name} VARCHAR(MAX), "
#     elif dtype == 'int64':
#         create_table_sql += f"{name} INT, "
#     elif dtype == 'float64':
#         create_table_sql += f"{name} FLOAT, "
#     elif dtype == 'bool':
#         create_table_sql += f"{name} BIT, "
#     elif dtype == 'datetime64[ns]':
#         create_table_sql += f"{name} DATETIME, "
# create_table_sql = create_table_sql[:-2] + ")"

# try:
#     #Createse the table in Azure SQL
#     engine.execute(create_table_sql)
#     print("Table",table_name,"succesfully created")
#     # Insert data into SQL Database
#     lower = 0
#     upper = 1000
#     limit = df.shape[0]

#     while lower < limit:
#         df[lower:upper].to_sql(table_name, con=engine, if_exists='append', index=False)
#         print("rows:", lower, "-", upper, "inserted")
#         lower = upper
#         upper = min(upper + 1000, limit)

# except Exception as e:
#     print(e)

# Query with LLM

In [47]:
# Create or LLM Langchain object using GPT-4 deployment
# Again we need GPT-4. It is necesary in the use of Agents. GPT-35-Turbo will make many mistakes.
llm = AzureChatOpenAI(deployment_name="gpt4", temperature=0, max_tokens=500)

In [48]:
#using a shorter prompt to handle token limitations
from langchain.prompts import PromptTemplate

MSSQL_PROMPT2 = """
You are an MS SQL expert. Given an input question, first create a syntactically correct MS SQL query to run, then look at the results of the query and return the answer to the input question.

Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the TOP clause as per MS SQL. You can order the results to return the most informative data in the database.
** ALWAYS before giving the Final Answer, try another method**. Then reflect on the answers of the two methods you did and ask yourself if it answers correctly the original question. If you are not sure, try another method.
If the runs does not give the same result, reflect and try again two more times until you have two runs that have the same result. If you still cannot arrive to a consistent result, say that you are not sure of the answer. But, if you are sure of the correct answer, create a beautiful and thorough response. DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE, ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE. 

Use the following format:

Question: How many people died of covid in Texas in 2020?
SQLQuery: SELECT [death] FROM covidtracking WHERE state = 'TX' AND date LIKE '2020%'
SQLResult: [(27437.0,), (27088.0,), (26762.0,), (26521.0,), (26472.0,), (26421.0,), (26408.0,)]
Answer: There were 27437 people who died of covid in Texas in 2020.

Only use the following tables:
{table_info}

Question: {input}"""

MSSQL_PROMPT2 = PromptTemplate(
    input_variables=["input", "table_info", "top_k"], 
    template=MSSQL_PROMPT2
)

In [49]:
# Let's use a type of Chain made for this type of SQL work.  
db = SQLDatabase.from_uri(db_url)
db_chain = SQLDatabaseChain(llm=llm, database=db, prompt=MSSQL_PROMPT2, verbose=False)

c:\source\.conda\lib\site-packages\langchain\chains\sql_database\base.py:63: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [ ]:
# Let's check our prompt we created in app/prompts.py
#print(db_chain.prompt.template)

In [52]:
# Natural Language question (query)
#QUESTION = "how many SOH In Stores for NSW for ANTISLIP TAPE SYNECO++48MM 20M BLK 1369, who is the supplier, and what is the next order due Date?"

QUESTION ="who are the suppliers for DOG HOUSE in NSW"

In [53]:
printmd(db_chain(QUESTION)['result'])

InvalidRequestError: This model's maximum context length is 8193 tokens. However, your messages resulted in 15536 tokens. Please reduce the length of the messages.

### To use or not use Agents

As you can see above we achieved our goal of Question->SQL without using an Agent, we did it just by using a clever prompt. (If you want to see the different kind of prompts templates that come with langchain for sql chain, you can check it out [HERE](https://github.com/hwchase17/langchain/blob/master/langchain/chains/sql_database/prompt.py)). **So the question is, why do we need an Agent then?**

**This is why**: As we explained on the prior Notebook, an agent is a process in which the LLM self-asks about what approach and steps to take, questions the validity of the results and if sure, provides the answer. The SQLDatabaseChain doesn't do all this analysis, but instead tries a one-shot query in order to answer the question, which is good! but not enough for complex questions. That's why it couldn't solve the part about nationwide, it needs multiple steps in order to solve the problem, one query is not enough.
Notice that it did't pay atention to the prompt where we explicitly say to try two methods and reflect on the answer.

Let's use an agent now and see how ReAct framework solves the problem.

In [12]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor = create_sql_agent(
    prefix=MSSQL_AGENT_PREFIX,
    format_instructions = MSSQL_AGENT_FORMAT_INSTRUCTIONS,
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

In [13]:
# As we know by now, Agents use expert/tools. Let's see which are the tools for this SQL Agent
agent_executor.agent.allowed_tools

['sql_db_query', 'sql_db_schema', 'sql_db_list_tables', 'sql_db_query_checker']

In [14]:
# And let's see now our clever crafted prompt
printmd(agent_executor.agent.llm_chain.prompt.template)



You are an agent designed to interact with a SQL database.
## Instructions:
- Given an input question, create a syntactically correct mssql query to run, then look at the results of the query and return the answer.
- Unless the user specifies a specific number of examples they wish to obtain, **ALWAYS** limit your query to at most 10 results.
- You can order the results by a relevant column to return the most interesting examples in the database.
- Never query for all the columns from a specific table, only ask for the relevant columns given the question.
- You have access to tools for interacting with the database.
- You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.
- DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
- DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE, ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE. 
- Your answer should be in Markdown.
- ALWAYS, as part of your final answer, explain how you got to the answer on a section that starts with: "Explanation:". Include the SQL query as part of the explanation section.
- If the question does not seem related to the database, just return "I don't know" as the answer.
- Only use the below tools. Only use the information returned by the below tools to construct your final answer.

## Tools:



sql_db_query: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', using schema_sql_db to query the correct table fields.
sql_db_schema: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling list_tables_sql_db first! Example Input: 'table1, table2, table3'
sql_db_list_tables: Input is an empty string, output is a comma separated list of tables in the database.
sql_db_query_checker: 
    Use this tool to double check if your query is correct before executing it.
    Always use this tool before executing a query with query_sql_db!
    



## Use the following format:

Question: the input question you must answer. 
Thought: you should always think about what to do. 
Action: the action to take, should be one of [sql_db_query, sql_db_schema, sql_db_list_tables, sql_db_query_checker]. 
Action Input: the input to the action. 
Observation: the result of the action. 
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer. 
Final Answer: the final answer to the original input question. 

Example of Final Answer:
<=== Beginning of example

Action: query_sql_db
Action Input: SELECT TOP (10) [death] FROM covidtracking WHERE state = 'TX' AND date LIKE '2020%'
Observation: [(27437.0,), (27088.0,), (26762.0,), (26521.0,), (26472.0,), (26421.0,), (26408.0,)]
Thought:I now know the final answer
Final Answer: There were 27437 people who died of covid in Texas in 2020.

Explanation:
I queried the `covidtracking` table for the `death` column where the state is 'TX' and the date starts with '2020'. The query returned a list of tuples with the number of deaths for each day in 2020. To answer the question, I took the sum of all the deaths in the list, which is 27437. 
I used the following query

```sql
SELECT [death] FROM covidtracking WHERE state = 'TX' AND date LIKE '2020%'"
```
===> End of Example



Begin!

Question: {input}
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
{agent_scratchpad}

In [15]:
for i in range(2):
    try:
        response = agent_executor.run(QUESTION) 
        break
    except Exception as e:
        response = str(e)
        continue



> Entering new  chain...
Action: sql_db_list_tables
Action Input: ""
Observation: covidtracking
Thought:I should check the schema of the covidtracking table to see what columns are available.
Action: sql_db_schema
Action Input: "covidtracking"
Observation: 
CREATE TABLE covidtracking (
	date VARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	state VARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	death FLOAT(53) NULL, 
	[deathConfirmed] FLOAT(53) NULL, 
	[deathIncrease] INTEGER NULL, 
	[deathProbable] FLOAT(53) NULL, 
	hospitalized FLOAT(53) NULL, 
	[hospitalizedCumulative] FLOAT(53) NULL, 
	[hospitalizedCurrently] FLOAT(53) NULL, 
	[hospitalizedIncrease] INTEGER NULL, 
	[inIcuCumulative] FLOAT(53) NULL, 
	[inIcuCurrently] FLOAT(53) NULL, 
	negative FLOAT(53) NULL, 
	[negativeIncrease] INTEGER NULL, 
	[negativeTestsAntibody] FLOAT(53) NULL, 
	[negativeTestsPeopleAntibody] FLOAT(53) NULL, 
	[negativeTestsViral] FLOAT(53) NULL, 
	[onVentilatorCumulative] FLOAT(53) NULL, 

In [16]:
printmd(response)

There were 0 patients hospitalized in Texas during July 2020. Nationwide, the total number of hospitalized patients during July 2020 was 65,732.

Explanation:
I queried the `covidtracking` table for the `hospitalizedIncrease` column where the date is between '2020-07-01' and '2020-07-31' for each state. The query returned a list of tuples with the state and the total number of hospitalized patients for each state during July 2020. To answer the question, I looked for the value corresponding to Texas (TX) and summed up all the values for the nationwide total. The result is 0 for Texas and 65,732 for nationwide. I used the following query:

```sql
SELECT state, SUM(hospitalizedIncrease) as total_hospitalized
FROM covidtracking
WHERE date >= '2020-07-01' AND date <= '2020-07-31'
GROUP BY state
```

This answer is way better than only using the SQL Chain

**IMPORTANT NOTE**: Runing the above cell multiple times still will yield diferent results some times. <br>
The reason is:
The column names are ambiguous, hence it is hard even for Humans to discern what are the right columns to use

# Summary

In this notebook, we achieved our goal of Asking a Question in natural language to a dataset located on a SQL Database.  We did this by using purely prompt engineering (Langchain does it for us) and the cognitive power of GPT-4.

This process shows why it is NOT necessary to move the data from its original source as long as the source has an API and a common language we can use to interface with. GPT-4 has been trained on the whole public Github corpus, so it can pretty much understand most of the coding and database query languages that exists out there. 

# NEXT

The Next Notebook will show you how to create a custom REACT agent that connects to the internet using BING SEARCH API to answer questions grounded on search results with citations. Basically a clone of Bing Chat.